# Conhecendo os dados

In [1]:
import pandas as pd

In [2]:
dados_churn = pd.read_json("dataset-telecon.json")
dados_churn.head()

FileNotFoundError: File dataset-telecon.json does not exist

In [ ]:
dados_churn['conta'][0]

In [ ]:
pd.json_normalize(dados_churn['conta']).head()

In [ ]:
pd.json_normalize(dados_churn['telefone']).head()

## Transformando dados em uma tabela

In [ ]:
import json

In [ ]:
with open("dataset-telecon.json") as f:
  json_bruto = json.load(f)

In [ ]:
json_bruto

In [ ]:
dados_normalizados = pd.json_normalize(json_bruto)
dados_normalizados.head()

# Transformação inicial dos dados


## Entendendo os dados

A base de dados contém colunas além do ID dos clientes e o churn:

<b>Cliente:</b>
 
* `genero`: gênero (masculino e feminino)
* `idoso`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos
* `parceiro`: se o cliente possui ou não um parceiro ou parceira
* `dependentes`: se o cliente possui ou não dependentes
* `tempo_servico`: meses de contrato do cliente

<b>Serviço de telefonia</b>

 * `servico_telefone`: assinatura de serviço telefônico
 * `varias_linhas`: assinatura de mais de uma linha de telefone
 

<b>Serviço de internet</b>


 * `servico_internet`: assinatura de um provedor internet
 * `seguranca_online`: assinatura adicional de segurança online
 * `backup_online`: assinatura adicional de backup online
 * `protecao_dispositivo`: assinatura adicional de proteção no dispositivo
 * `suporte_tecnico`: assinatura adicional de suporte técnico, menos tempo de espera
 * `tv_streaming`: assinatura de TV a cabo
 * `filmes_streaming`: assinatura de streaming de filmes


<b>Conta</b>

 * `contrato`: tipo de contrato
 * `faturamente_eletronico`: se o cliente prefere receber online a fatura
 * `metodo_pagamento`: forma de pagamento
 * `cobranca.mensal`: total de todos os serviços do cliente por mês
 * `cobranca.Total`: total gasto pelo cliente

In [ ]:
dados_normalizados.info()

In [ ]:
dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)

## Modificando o tipo da coluna

In [ ]:
dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '].head()

In [ ]:
dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '][
    ['cliente.tempo_servico', 'conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']
]

In [ ]:
idx = dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '].index

In [ ]:
dados_normalizados.loc[idx, "conta.cobranca.Total"] = dados_normalizados.loc[idx, "conta.cobranca.mensal"] * 24

In [ ]:
dados_normalizados.loc[idx, "cliente.tempo_servico"] = 24

In [ ]:
dados_normalizados.loc[idx][
    ['cliente.tempo_servico', 'conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']
]

In [ ]:
dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)

In [ ]:
dados_normalizados.info()

## Identificando e tratando strings vazias

In [ ]:
for col in dados_normalizados.columns:
    print(f"Coluna: {col}")
    print(dados_normalizados[col].unique())
    print("-" * 30)

In [ ]:
dados_normalizados.query("Churn == ''")

In [ ]:
dados_sem_vazio = dados_normalizados[dados_normalizados['Churn'] != ''].copy()

In [ ]:
dados_sem_vazio.info()

In [ ]:
dados_sem_vazio.reset_index(drop=True, inplace=True)

In [ ]:
dados_sem_vazio

# Trabalhando com dados duplicados e nulos

## Identificando e tratando dados duplicados


In [ ]:
dados_sem_vazio.duplicated()

In [ ]:
dados_sem_vazio.duplicated().sum()

In [ ]:
filtro_duplicadas = dados_sem_vazio.duplicated()
filtro_duplicadas

In [ ]:
dados_sem_vazio[filtro_duplicadas]

In [ ]:
dados_sem_vazio.drop_duplicates(inplace=True)

In [ ]:
dados_sem_vazio.duplicated().sum()

## Identificando e substituindo dados nulos

In [ ]:
dados_sem_vazio.isna()

In [ ]:
dados_sem_vazio.isna().sum()

In [ ]:
dados_sem_vazio.isna().sum().sum()

In [ ]:
dados_sem_vazio[dados_sem_vazio.isna().any(axis=1)]

In [ ]:
filtro = dados_sem_vazio['cliente.tempo_servico'].isna()

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
import numpy as np

np.ceil(5957.90/90.45)

In [ ]:
dados_sem_vazio['cliente.tempo_servico'].fillna(
    np.ceil(
        dados_sem_vazio['conta.cobranca.Total'] / dados_sem_vazio['conta.cobranca.mensal']
    ), inplace=True
)

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
dados_sem_vazio.isna().sum()

## Retirando os dados nulos

In [ ]:
dados_sem_vazio['conta.contrato'].value_counts()

In [ ]:
colunas_dropar = ['conta.contrato', 'conta.faturamente_eletronico', 'conta.metodo_pagamento']

In [ ]:
dados_sem_vazio[colunas_dropar].isna().any(axis=1).sum()

In [3]:
df_sem_nulo = dados_sem_vazio.dropna(subset=colunas_dropar).copy()
df_sem_nulo.head()

NameError: name 'dados_sem_vazio' is not defined

In [4]:
df_sem_nulo.reset_index(drop=True, inplace=True)

NameError: name 'df_sem_nulo' is not defined

In [5]:
df_sem_nulo.isna().sum()

NameError: name 'df_sem_nulo' is not defined

# Trabalhando com os outliers

## Identificando os outliers

![](https://i.imgur.com/Arb5cXL.png)

In [6]:
df_sem_nulo.describe()

NameError: name 'df_sem_nulo' is not defined

In [7]:
1080/12

90.0

In [8]:
import seaborn as sns

In [9]:
sns.boxplot(x=df_sem_nulo['cliente.tempo_servico'])

NameError: name 'df_sem_nulo' is not defined

In [10]:
Q1 = df_sem_nulo['cliente.tempo_servico'].quantile(.25)
Q3 = df_sem_nulo['cliente.tempo_servico'].quantile(.75)
IQR = Q3-Q1
limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR

NameError: name 'df_sem_nulo' is not defined

In [11]:
outliers_index = (df_sem_nulo['cliente.tempo_servico'] < limite_inferior) | (df_sem_nulo['cliente.tempo_servico'] > limite_superior)
outliers_index

NameError: name 'df_sem_nulo' is not defined

In [12]:
df_sem_nulo[outliers_index]['cliente.tempo_servico']

NameError: name 'df_sem_nulo' is not defined

## Substituindo valores para os outliers

In [13]:
df_sem_out = df_sem_nulo.copy()

NameError: name 'df_sem_nulo' is not defined

In [14]:
df_sem_out[outliers_index]['cliente.tempo_servico']

NameError: name 'df_sem_out' is not defined

In [15]:
df_sem_out.loc[outliers_index, 'cliente.tempo_servico'] = np.ceil(
    df_sem_out.loc[outliers_index, 'conta.cobranca.Total'] /
    df_sem_out.loc[outliers_index, 'conta.cobranca.mensal']
)

NameError: name 'np' is not defined

In [16]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

NameError: name 'df_sem_out' is not defined

In [17]:
df_sem_out[outliers_index][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

NameError: name 'df_sem_out' is not defined

## Removendo outliers

In [18]:
df_sem_out[outliers_index]['cliente.tempo_servico']

NameError: name 'df_sem_out' is not defined

In [19]:
Q1 = df_sem_out['cliente.tempo_servico'].quantile(.25)
Q3 = df_sem_out['cliente.tempo_servico'].quantile(.75)
IQR = Q3-Q1
limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR

outliers_index = (df_sem_out['cliente.tempo_servico'] < limite_inferior) | (df_sem_out['cliente.tempo_servico'] > limite_superior)
outliers_index

NameError: name 'df_sem_out' is not defined

In [20]:
df_sem_out[outliers_index]

NameError: name 'df_sem_out' is not defined

In [21]:
df_sem_out = df_sem_out[~outliers_index]
df_sem_out

NameError: name 'df_sem_out' is not defined

In [22]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

NameError: name 'df_sem_out' is not defined

In [23]:
df_sem_out.reset_index(drop=True, inplace=True)

NameError: name 'df_sem_out' is not defined

In [24]:
df_sem_out

NameError: name 'df_sem_out' is not defined

# Trabalhando com variáveis categóricas

## Substituindo valores

In [25]:
df_sem_id = df_sem_out.drop('id_cliente', axis=1).copy()
df_sem_id

NameError: name 'df_sem_out' is not defined

In [26]:
mapeamento = {
    'nao': 0,
    'sim': 1,
    'masculino': 0,
    'feminino': 1
}

In [27]:
for col in df_sem_id.columns:
    print(f"Coluna: {col}")
    print(df_sem_id[col].unique())
    print("-" * 30)

NameError: name 'df_sem_id' is not defined

In [28]:
colunas = ['telefone.servico_telefone', 'Churn', 'cliente.parceiro', 'cliente.dependentes', 'conta.faturamente_eletronico', 'cliente.genero']

In [29]:
df_sem_id[colunas] = df_sem_id[colunas].replace(mapeamento)
df_sem_id

NameError: name 'df_sem_id' is not defined

In [30]:
for col in df_sem_id.columns:
    print(f"Coluna: {col}")
    print(df_sem_id[col].unique())
    print("-" * 30)

NameError: name 'df_sem_id' is not defined

## One Hot Encoder (dummy)

In [31]:
s = pd.Series(list('abca'))
s

0    a
1    b
2    c
3    a
dtype: object

In [32]:
pd.get_dummies(s, dtype=int)

,a,b,c
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0


In [33]:
df_sem_id.info()

NameError: name 'df_sem_id' is not defined

In [34]:
df_dummies = pd.get_dummies(df_sem_id, dtype=int).copy()
df_dummies.head()

NameError: name 'df_sem_id' is not defined

In [35]:
df_dummies.columns

NameError: name 'df_dummies' is not defined

In [36]:
df_dummies.info()

NameError: name 'df_dummies' is not defined